In [1]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
X = pd.read_csv('i.csv', header = None, float_precision='high', sep = ';', dtype = np.float64)
y = pd.read_csv('o.csv', header = None, float_precision='high', sep = ';', dtype = np.float64)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
reg = make_pipeline(StandardScaler(), SGDRegressor(tol = 1e-4, learning_rate='optimal', eta0 = 0.001, max_iter=10000000))
reg.fit(X_train, np.ravel(y_train))

In [ ]:
predictions = reg.predict(X_test)
mse = mean_squared_error(y_test, predictions) 
print("RMSE: {}".format(np.sqrt(mse)))